In [3]:
from ultralytics import YOLO
import os


frames_dir = "frames"
labels_dir = "labels"

os.makedirs(labels_dir, exist_ok=True)

model = YOLO("yolov8n.pt") 

keep_classes = ["car", "bus", "truck", "motorcycle"]

model_classes = model.names
keep_ids = [i for i, name in model_classes.items() if name in keep_classes]

print("classs:", [model_classes[i] for i in keep_ids])

for img_name in os.listdir(frames_dir):
    if not img_name.lower().endswith(".jpg"):
        continue

    img_path = os.path.join(frames_dir, img_name)
    results = model(img_path, verbose=False)


    detections = results[0].boxes
    boxes = detections.xywhn.cpu().numpy()  # [x_center, y_center, w, h] normalized
    cls = detections.cls.cpu().numpy()

    # Lọc class
    output_lines = []
    for b, c in zip(boxes, cls):
        if int(c) in keep_ids:
            output_lines.append(f"{int(c)} {b[0]:.6f} {b[1]:.6f} {b[2]:.6f} {b[3]:.6f}")

    # Lưu ra file txt cùng tên
    label_path = os.path.join(labels_dir, img_name.replace(".jpg", ".txt"))
    with open(label_path, "w") as f:
        f.write("\n".join(output_lines))

print('finished')



classs: ['car', 'motorcycle', 'bus', 'truck']
finished


In [ ]:
from ultralytics import YOLO
import os
import numpy as np

frames_dir = "frames"
labels_dir = "labels2"

os.makedirs(labels_dir, exist_ok=True)
model = YOLO("best.pt")

keep_classes = ["car", "bus", "truck", "motorcycle"]
model_classes = model.names
keep_ids = [i for i, name in model_classes.items() if name in keep_classes]

for img_name in os.listdir(frames_dir):
    if not img_name.lower().endswith(".jpg"):
        continue

    img_path = os.path.join(frames_dir, img_name)
    results = model(img_path, verbose=False)

    detections = results[0].boxes
    keypoints = results[0].keypoints  # lấy keypoints nếu có

    boxes = detections.xywhn.cpu().numpy() if detections is not None else []
    cls = detections.cls.cpu().numpy() if detections is not None else []

    output_lines = []
    for i, (b, c) in enumerate(zip(boxes, cls)):
        if int(c) not in keep_ids:
            continue

        # tọa độ bbox
        line = [int(c), b[0], b[1], b[2], b[3]]

        if keypoints is not None and len(keypoints.data) > i:
            kpts = keypoints.xyn[i].cpu().numpy().flatten()
            line.extend(kpts)

        num_kpts = len(kpts) // 2 if keypoints is not None else 0
        line.append(num_kpts)

        # định dạng ghi ra file
        output_lines.append(" ".join(f"{x:.6f}" if isinstance(x, float) else str(x) for x in line))

    # ghi file txt
    label_path = os.path.join(labels_dir, img_name.replace(".jpg", ".txt"))
    with open(label_path, "w") as f:
        f.write("\n".join(output_lines))

print("✅ Done generating label files!")


✅ Done generating label files!
